In [1]:
import getpass
import helm

from helm.common.authentication import Authentication
from helm.common.perspective_api_request import PerspectiveAPIRequest, PerspectiveAPIRequestResult
from helm.common.request import Request, RequestResult
from helm.common.tokenization_request import TokenizationRequest, TokenizationRequestResult
from helm.proxy.accounts import Account

#google, gpt3, gpt4, anthropic and together
from helm.proxy.services.remote_service import RemoteService

import numpy as np
import pandas as pd
from tqdm import tqdm
import time

In [2]:
#enter the key here
key = ""

api_key = getpass.getpass(prompt="Enter a valid API key: ")
auth = Authentication(api_key=key)
service = RemoteService("https://crfm-models.stanford.edu")

# Access account and show my current quotas and usages
account: Account = service.get_account(auth)
print(account.usages.keys())

for key in account.usages.keys():
    print(key, account.usages[key])

Enter a valid API key:  ········


dict_keys(['gpt3', 'gpt4', 'codex', 'jurassic', 'gooseai', 'cohere', 'dall_e', 'together_vision', 'simple', 'together', 'anthropic', 'google', 'o1'])
gpt3 {'daily': Usage(period='2024-12-22', used=301, quota=None), 'monthly': Usage(period='2024-12', used=301, quota=None), 'total': Usage(period='all', used=1325509, quota=10000000)}
gpt4 {'daily': Usage(period='2024-12-22', used=290, quota=None), 'monthly': Usage(period='2024-12', used=290, quota=None), 'total': Usage(period='all', used=1443087, quota=10000000)}
codex {'daily': Usage(period=None, used=0, quota=10000)}
jurassic {'daily': Usage(period=None, used=0, quota=10000)}
gooseai {'daily': Usage(period=None, used=0, quota=10000)}
cohere {'daily': Usage(period=None, used=0, quota=10000)}
dall_e {'daily': Usage(period=None, used=0, quota=5)}
together_vision {'daily': Usage(period=None, used=0, quota=30)}
simple {'daily': Usage(period=None, used=0, quota=10000)}
together {'daily': Usage(period='2024-12-22', used=300, quota=None), 'mont

In [3]:
df = pd.read_csv('pairs_metadata.csv')

In [4]:
valid_llms = ['openai/o1-preview-2024-09-12','anthropic/claude-3-5-sonnet-20240620',
              'openai/gpt-3.5-turbo-0125',
              'openai/gpt-4o-2024-05-13',
              'meta/llama-3.1-70b-instruct-turbo',
             'google/gemini-1.5-pro-001']

In [5]:
with open("prompt.txt", "r", encoding="utf-8") as file:
    prompt = file.read()

def prompt_llm(prompt, llm):
    if llm == 'openai/o1-preview-2024-09-12':
        mx_tokens = 32768
        request = Request(
            model=llm, prompt=prompt, echo_prompt=False,
            max_tokens=mx_tokens,
        )
    else:
        request = Request(
            model=llm, prompt=prompt, echo_prompt=False
        )
    request_result: RequestResult = service.make_request(auth, request)
    return request_result.completions[0].text

In [6]:
list_outputs = []
for _,row in tqdm(df.iterrows()):

    entry = {}
    for llm in valid_llms:
        
        input_text = prompt + "Recipe 1:\n"+row['text_1'] + "\n\nRecipe 2:\n"+row['text_2']+ "\n\nAnswer:"
        
        try:
            entry[llm] = prompt_llm(input_text, llm)
            time.sleep(0.5)
        except:
            print('failed')
            entry[llm] = np.nan
    list_outputs.append(entry)  

24it [04:15, 11.42s/it]

failed


500it [1:27:19, 10.48s/it]


In [7]:
pd.DataFrame(list_outputs).to_csv('collected_outputs.csv')